## Scrapy
- 비동기 방식으로 데이터를 수집 > 데이터 수집 속도가 빠름
- 동기 방식) (rq - nt - rp) 시작한 수집을 끝내야 다음 수집을 시작할 수 있음.
- 비동기 방식) nt을 기다리는 동안, 2번 rq시작.
    - 네트워크 타임을 절약.

In [1]:
import scrapy, requests
from scrapy.http import TextResponse

In [2]:
# 1. 스크래피 프로젝트 생성
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\KT_AIVLE\3_Web_Crawling\github_upload\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [3]:
!tree gmarket /f

폴더 PATH의 목록입니다.
볼륨 일련 번호가 000000C0 7C6C:FC35입니다.
C:\USERS\USER\KT_AIVLE\3_WEB_CRAWLING\GITHUB_UPLOAD\GMARKET
│  scrapy.cfg
│  
└─gmarket
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


- items.py : 수집할 데이터의 구조 정의
- middlewares.py : 데이터를 수집할때 headers 정보와 같은 내용을 설정
- pipelines.py : 데이터를 수집한 후에 코드를 실행 정의
- settings.py : 크롤링 설정 : 크롤링 시간 텀, robots.txt 규칙 수행 여부
- spiders : 디렉토리 : 크롤링 절차 정의

In [4]:
# 2. xpath 찾기: 링크, 상세 페이지의 데이터
url = "http://corners.gmarket.co.kr/Bestsellers"
request = requests.get(url)
response = TextResponse(request.url, body=request.text, encoding="utf-8")
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href') # href 가져오기
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract() # link 추출
link = links[0] # 상세 페이지 데이터 수집

In [5]:
# 3. TextResponse: 가져온 웹페이지를 돔구조로 만드는데 사용.
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding="utf-8")

In [6]:
# 4. 뒤에 text() 를 붙여서 텍스트 데이터만 추출.
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()
title, price

('프리미어 70매 캡 20팩  70gsm ', '28,900')

In [7]:
# 5. item.py: 코드 작성: model(데이터의 구조: 수집할 데이터의 컬럼을 정의)

In [8]:
%%writefile gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [10]:
# 6. spider.py : 코드 작성 : 크롤링 절차 정의
# - name : spider의 이름.
# - allow_domain : 해당 페이지만 크롤링.
# - start_urls : 최초로 request response를 하는 url.
    # - links : link 데이터를 가져옴.

In [11]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content)
    
    # 위 self.parse_content와 식별자를 맞춰주어야 한다.
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Writing gmarket/gmarket/spiders/spider.py


In [12]:
# 7. 스크래피 프로젝트 실행
# - gmarket 디렉토리
#- scrapy crawl GMB -o items.csv
   # - -o : output

In [13]:
%pwd

'C:\\Users\\User\\KT_AIVLE\\3_Web_Crawling\\github_upload'